### 네이버영화에서 각 영화 상세정보 링크 가져오기

In [225]:
import requests
from bs4 import BeautifulSoup

url = "https://movie.naver.com/movie/running/current.nhn"

url_req = requests.get(url)

url_soup = BeautifulSoup(url_req.text)

movie_link_list = []

link_soup = url_soup.select("div.thumb > a")

for link in link_soup:
    link = link.get('href')
    
    movie_link_list.append('https://movie.naver.com' + link)
movie_link_list

soup = url_soup.select("div.star_t1.b_star > span")

advanceRate = []

for i in range(len(soup)):
    if i % 2 == 0:
        advanceRate.append(float(soup[i].text))
        
for i in range(6):
    advanceRate.append(0.00)

### 각 링크에서 세부정보를 가져와서 dataframe으로 return 해주는 함수

In [349]:
import pandas as pd
import random
from tqdm import tqdm

def getMovieDataFromNaverMovie(movie_link_list):
    
    titles = []
    posterUrls = []
    screeningStatuss = []
    advanceRates = []
    advancdRateRank = []
    visitorRatings = []
    expertRatings = []
    plots = []
    movie_genres = []
    runtimes = []
    casts = []
    releaseds = []
    mv_directors = []
    mv_actors = []
            
    for i in tqdm(range(len(movie_link_list))):
        try:
            url_req = requests.get(movie_link_list[i])

            url_soup = BeautifulSoup(url_req.text)

            title = url_soup.select('div.mv_info > h3.h_movie > a')[0].text

            genres = []

            genre_soup = url_soup.select('dl.info_spec > dd > p > span:nth-child(1)> a')

            for genre in genre_soup:
                genres.append(genre.text)

            runtime = int(re.sub("[^0-9]", '' , url_soup.select('dl.info_spec > dd > p > span:nth-child(3)')[0].text))

            try:
                release_date = url_soup.select('dl.info_spec > dd > p > span:nth-child(4) > a')[0].text + url_soup.select('dl.info_spec > dd > p > span:nth-child(4) > a')[1].text.replace(".", "-")
            except:
                release_date = '2020-01-30'

            director = url_soup.select('dl.info_spec > dd:nth-child(4) > p > a')

            directors = []

            for direct in director:
                directors.append(direct.text)

            actors = []

            actor_soup = url_soup.select('dl.info_spec > dd:nth-child(6) > p > a')

            for actor in actor_soup:
                actors.append(actor.text)

            plot_soup = url_soup.select('p.con_tx')
            plot = plot_soup[0].text.replace("\r", "").replace("\xa0", "").replace('"', '').replace("'", "")
            plot = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]', '', plot)

            star_score = url_soup.select('div.star_score > em')

            if len(star_score) == 15:
                star_score_1 = float(star_score[8].text)
                star_score_2 = float(star_score[9].text)
            elif len(star_score) == 22:
                star_score_1 = float(star_score[16].text)
                star_score_2 = float(star_score[17].text)
            else:
                star_score_1 = float(random.randint(5, 10))
                star_score_2 = float(random.randint(5, 10))
            
            movie_id = re.sub("[^0-9]", '', movie_link_list[i])

            poster_url = "https://movie.naver.com/movie/bi/mi/photoViewPopup.nhn?movieCode="+ movie_id

            poster_req = requests.get(poster_url)
            poster_soup = BeautifulSoup(poster_req.text)
            poster_link = poster_soup.select("img#targetImage")[0].get('src')
            
            titles.append(title)
            posterUrls.append(poster_link)
            screeningStatuss.append("open")
            advanceRates.append(advanceRate[i])
            advancdRateRank.append(i+1)
            visitorRatings.append(star_score_1)
            expertRatings.append(star_score_2)
            plots.append(plot)
            movie_genres.append(genres)
            runtimes.append(runtime)
            casts.append(actors)
            releaseds.append(release_date)
            mv_directors.append(directors[0])

        except:
            print("error : ", i)
        
    
    my_dict = {"title":titles, "posterUrl":posterUrls, "screeningStatus":screeningStatuss, "advanceRate":advanceRates, "advanceRateRank":advancdRateRank, "visitorRating":visitorRatings, "expertRatings":expertRatings, "plot":plots, "genres":movie_genres, "runtime":runtimes, "cast":casts, "release":releaseds, "director":mv_directors}

    my_dataFrame = pd.DataFrame(my_dict)
    return my_dataFrame

In [350]:
getMovieDataFromNaverMovie(movie_link_list).to_csv("movie_data5.csv", index=False)

100%|██████████| 73/73 [00:26<00:00,  2.32it/s]


In [351]:
pd.read_csv("movie_data5.csv")

,title,posterUrl,screeningStatus,advanceRate,advanceRateRank,visitorRating,expertRatings,plot,genres,runtime,cast,release,director
0,인비저블맨,https://movie-phinf.pstatic.net/20200221_216/1...,open,24.78,1,8.00,9.0,모든 것을 통제하려는 소시오패스 남자에게서 도망친 세실리아그의 자살 소식과 함께 상...,"['공포', '스릴러']",124,['엘리자베스 모스'],2020-02-26,리 워넬
1,지푸라기라도 잡고 싶은 짐승들,https://movie-phinf.pstatic.net/20200211_156/1...,open,21.14,2,10.00,7.0,모든 것은 돈 가방과 함께 시작되었다사라진 애인 때문에 사채 빚에 시달리며 한 탕을...,"['범죄', '스릴러']",108,"['전도연', '정우성', '배성우']",2020-02-19,김용훈
2,1917,https://movie-phinf.pstatic.net/20200212_161/1...,open,19.19,3,10.00,10.0,제1차 세계대전이 한창인 1917년독일군에 의해 모든 통신망이 파괴된 상황 속에서영...,"['드라마', '전쟁']",119,"['조지 맥케이', '딘-찰스 채프먼']",2020-02-19,샘 멘데스
3,정직한 후보,https://movie-phinf.pstatic.net/20200121_186/1...,open,10.54,4,10.00,7.0,거짓말이 제일 쉬운 3선 국회의원 주상숙’에게 청천벽력이 떨어진다하루아침에 거짓말은...,['코미디'],104,"['라미란', '김무열', '나문희']",2020-02-12,장유정
4,작은 아씨들,https://movie-phinf.pstatic.net/20200117_184/1...,open,8.87,5,8.00,8.0,배우가 되고 싶은 첫째 메그엠마 왓슨작가가 되고 싶은 둘째 조시얼샤 로넌음악가가 되...,"['드라마', '멜로/로맨스']",135,"['시얼샤 로넌', '엠마 왓슨', '플로렌스 퓨']",2020-02-12,그레타 거윅
5,젠틀맨,https://movie-phinf.pstatic.net/20200211_164/1...,open,5.37,6,9.00,9.0,유럽을 장악한 업계의 절대강자 믹키 피어슨’매튜 맥커너히은자신이 세운 마리화나 제국...,"['범죄', '액션']",113,"['매튜 맥커너히', '휴 그랜트', '콜린 파렐']",2020-02-26,가이 리치
6,시원찮은 그녀를 위한 육성방법 피날레,https://movie-phinf.pstatic.net/20200115_83/15...,open,1.77,7,8.24,0.0,어느 봄날 아키 토모야는 벚꽃이 흩날리는 언덕길에서 운명적으로 만난 소녀 카토 메구...,"['애니메이션', '드라마']",114,"['마츠오카 요시츠구', '야스노 키요노', '오오니시 사오리']",2020-02-27,카메이 칸타
7,장마,https://movie-phinf.pstatic.net/20200205_84/15...,open,1.37,8,9.71,0.0,북한에서 북송北送 재일교포의 아들로 태어나 굶주림과 인권 탄압에 시달리다자유를 찾아...,['다큐멘터리'],57,['이은택'],2020-02-27,이용남
8,조조 래빗,https://movie-phinf.pstatic.net/20200212_4/158...,open,1.02,9,6.00,8.0,제2차 세계대전 말기 엄마 로지’스칼렛 요한슨와 단둘이 살고 있는 10살 소년 조조...,"['코미디', '드라마', '전쟁']",108,"['스칼렛 요한슨', '로만 그리핀 데이비스', '타이카 와이티티']",2020-02-05,타이카 와이티티
9,클로젯,https://movie-phinf.pstatic.net/20200116_23/15...,open,1.02,10,6.00,9.0,갑작스러운 사고로 아내를 잃은 상원하정우과 그의 딸 이나허율상원은 소원해진 이나와의...,"['미스터리', '드라마']",98,"['하정우', '김남길', '허율']",2020-02-05,김광빈


In [352]:
movie_data = pd.read_csv("movie_data5.csv")

In [ ]:
json_template = {
  "title": movie_data['title'].iloc[i],
  "posterUrl": movie_data['posterUrl'].iloc[i],
  "screeningStatus": "open",
  "advanceRate": movie_data['advanceRate'].iloc[i],
  "advanceRateRank": movie_data['advanceRateRank'].iloc[i],
  "visitorRating": movie_data['visitorRating'].iloc[i],
  "expertRating": movie_data['expertRatings'].iloc[i],
  "plot": movie_data['plot'].iloc[i],
  "genres": movie_data['genres'].iloc[i],
  "runtime": movie_data['runtime'].iloc[i],
  "cast": movie_data['cast'].iloc[i],
  "released": movie_data['release'].iloc[i],
  "director": movie_data['director'].iloc[i]
}

### 데이터 형식을 json 형태의 string으로 만들어 return 해주는 함수

In [295]:
import json

def makeJsonList(movie_data):
    json_list = []
    for i in tqdm(range(len(movie_data['title']))):
        json_template = {
          "title": list(movie_data['title'])[i],
          "posterUrl": list(movie_data['posterUrl'])[i],
          "screeningStatus": "open",
          "advanceRate": list(movie_data['advanceRate'])[i],
          "advanceRateRank": list(movie_data['advanceRateRank'])[i],
          "visitorRating": list(movie_data['visitorRating'])[i],
          "expertRating": list(movie_data['expertRatings'])[i],
          "plot": list(movie_data['plot'])[i],
          "genres": list(movie_data['genres'])[i],
          "runtime": list(movie_data['runtime'])[i],
          "cast": list(movie_data['cast'])[i],
          "released": list(movie_data['release'])[i],
          "director": list(movie_data['director'])[i]
        }
        
        json_data = str(json_template)
        json_list.append(json_data.replace("'", '"').replace('"[', '[').replace(']"', "]"))

    return json_list

In [346]:
json_list = makeJsonList(movie_data)
test = json_list[5].replace("'", '"').replace('"[', '[').replace(']"', "]")
test

100%|██████████| 73/73 [00:00<00:00, 2030.37it/s]


'{"title": "젠틀맨", "posterUrl": "https://movie-phinf.pstatic.net/20200211_164/1581396524128hxWV1_JPEG/movie_image.jpg", "screeningStatus": "open", "advanceRate": 5.37, "advanceRateRank": 6, "visitorRating": 8.0, "expertRating": 9.0, "plot": "유럽을 장악한 업계의 절대강자 ‘믹키 피어슨’(매튜 맥커너히)은자신이 세운 마리화나 제국을 걸고 돈이라면 무엇이든 벌이는미국의 억만장자와의 빅딜을 시작한다.소문을 듣고 찾아온 무법자 ‘드라이 아이’(헨리 골딩)와돈 냄새를 맡은 사립탐정 ‘플레처’(휴 그랜트)까지 게임에 끼어들게 되면서오랫동안 지켜온 정글의 질서는 점점 무너지기 시작하는데…", "genres": ["범죄", "액션"], "runtime": 113, "cast": ["매튜 맥커너히", "휴 그랜트", "콜린 파렐"], "released": " 2020-02-26", "director": "가이 리치"}'

In [353]:
json_list = makeJsonList(movie_data)
test = json_list[6]
test

100%|██████████| 73/73 [00:00<00:00, 2424.20it/s]


'{"title": "시원찮은 그녀를 위한 육성방법 피날레", "posterUrl": "https://movie-phinf.pstatic.net/20200115_83/1579075630892Dh10J_JPEG/movie_image.jpg", "screeningStatus": "open", "advanceRate": 1.77, "advanceRateRank": 7, "visitorRating": 8.24, "expertRating": 0.0, "plot": "어느 봄날 아키 토모야는 벚꽃이 흩날리는 언덕길에서 운명적으로 만난 소녀 카토 메구미를 메인 히로인으로 삼아서 동인 게임을 제작하기로 마음을 먹는다 미술부 소속이지만 동인 일러스트레이터로도 활동 중인 사와무라 스펜서 에리리’ 우등생이면서 라이트노벨 작가로도 활약하고 있는 3학년 선배 카스미가오카 우타하’를 영입한 그는 게임 제작 동아리 블레싱 소프트웨어’를 결성해서 시행착오 끝에 첫 작품을 발표한다에리리와 우타하는 인기 크리에이터 코사카 아카네가 기획을 맡은 대작 게임 필즈 크로니클의 개발 멤버로 스카우트된다 한편 블레싱 소프트웨어 대표인 토모야는 서클 활동을 이어 나가면서 일러스트레이터인 하시마 이즈미 프로듀서인 하시마 이오리 음악 밴드 아이시 테일의 효도 미치루 그리고 부대표 메구미와 함께 신작 게임의 개발을 시작하는데 에리리와 우타하의 대작은 어떻게 될 것인가토모야와 메구미의 관계에 변화가과연 블레싱 소프트웨어의 신작의 행방은시원찮은 히로인을 둘러싼청춘 그래피티 그랜드 피날레", "genres": ["애니메이션", "드라마"], "runtime": 114, "cast": ["마츠오카 요시츠구", "야스노 키요노", "오오니시 사오리"], "released": " 2020-02-27", "director": "카메이 칸타"}'

In [376]:
import requests 
import json 
url = "http://54.180.149.147:8080/api/team/2/movie" 
req = requests.get(url) 
json = req.json()
json

[{'genres': ['드라마'],
  'cast': ['브루노 간츠'],
  '_id': '5e5cad18168330138949b620',
  'title': '영원과 하루',
  'posterUrl': 'https://movie-phinf.pstatic.net/20111222_262/1324542499836vf90S_JPEG/movie_image.jpg',
  'screeningStatus': 'open',
  'advanceRate': 0,
  'advanceRateRank': 73,
  'visitorRating': 9,
  'expertRating': 7,
  'plot': '데오 앙겔로풀로스의 11번째 영화 98 칸느 영화제 황금종려상 수상작 안개 낀 도시 테살로니키 거리는 소음으로 시끄럽고 해변가 공원은 검은 옷을 입은 노인들로 가득하다 이 황량하고 쓸쓸한 도시의 낡은 집에서 초로의 알렉산더는 외롭게 죽어가고 있다 그리스의 유명한 시인으로 존경받는 그이지만 죽음 앞에서 지난 세월은 덧없이 무상하기만 하다 병원에 들어가야 하지만 마지막 생의 순간을 병원에서 보내고 싶지 않은 알렉산더 그는 그에게 남겨진 하루를 평생의 숙업인 19세기 시인 솔로모스의 흩어진 시어들을 찾는 여행으로 보내고자 한다 우연히 짐을 정리하다가 30년 전 아내 안나가 쓴 편지를 찾게 되면서 그의 여행은 과거와 현실 기억과 환상이 교차하는 신비스러운 여행이 되어간다 편지는 아내와 가족 친구들과 함께 보냈던 눈부시게 아름다웠던 하루로 그를 데려가고 회색빛 절망의 현실과 눈부신 햇살로 가득한 과거를 오가고 알렉산더는 젊은 시절 일에만 매달려 아내를 외롭게 했던 것을 깨닫게 된다 왜 그때는 사랑하는 법을 몰랐을까 알렉산더의 때늦은 후회는 다시 한번 그를 절망 속에 빠뜨린다 여행 중인 알렉산더에게 또 하나의 깨달음은 알바니아 난민 소년이 전해준 솔로모스의 시어들을 통해 주어진다 그것은 바로 사랑하는 사람들과 보낸 시간이 그 어떤 위대한 시어보다 아름답고 영원하다는 것 그토록 찾아헤매인 

In [377]:
for i in range(len(json)): 
    print("제목 : ", json[i]['title']) 
    print("장르 : ", json[i]['genres']) 
    print("개봉일 : ", json[i]['released']) 

제목 :  영원과 하루
장르 :  ['드라마']
개봉일 :  2004-11-18T15:00:00.000Z
제목 :  비투스
장르 :  ['드라마']
개봉일 :  2008-04-08T15:00:00.000Z
제목 :  베를린 천사의 시
장르 :  ['판타지', '드라마', '멜로/로맨스']
개봉일 :  1993-05-14T15:00:00.000Z
제목 :  미스 사이공: 25주년 특별 공연
장르 :  ['공연실황']
개봉일 :  2016-11-23T15:00:00.000Z
제목 :  더 파티
장르 :  ['코미디', '드라마']
개봉일 :  2018-12-19T15:00:00.000Z
제목 :  다운폴
장르 :  ['전쟁', '드라마']
개봉일 :  2014-01-22T15:00:00.000Z
제목 :  혼 : 공포의 시작
장르 :  ['공포']
개봉일 :  2020-02-12T15:00:00.000Z
제목 :  파리로 가는 길
장르 :  ['드라마', '멜로/로맨스']
개봉일 :  2017-08-02T15:00:00.000Z
제목 :  주전장
장르 :  ['다큐멘터리']
개봉일 :  2019-07-24T15:00:00.000Z
제목 :  조커
장르 :  ['스릴러', '드라마']
개봉일 :  2019-10-01T15:00:00.000Z
제목 :  울지마 톤즈
장르 :  ['다큐멘터리']
개봉일 :  2010-09-08T15:00:00.000Z
제목 :  몽마르트 파파
장르 :  ['다큐멘터리']
개봉일 :  2020-01-08T15:00:00.000Z
제목 :  개인교습
장르 :  ['멜로/로맨스']
개봉일 :  2020-03-01T15:00:00.000Z
제목 :  작은 빛
장르 :  ['드라마']
개봉일 :  2020-01-22T15:00:00.000Z
제목 :  졸업
장르 :  ['드라마', '코미디']
개봉일 :  2020-02-13T00:00:00.000Z
제목 :  미스터 주: 사라진 VIP
장르 :  ['코미디', '드라마']
개봉일 :  2020

In [239]:
requests.post(url, data=json_list[3])

<Response [200]>

In [241]:
for i in range(len(json)): 
    print("제목 : ", json[i]['title']) 
    print("장르 : ", json[i]['genres']) 
    print("개봉일 : ", json[i]['released']) 

제목 :  기생충
장르 :  ['드라마']
개봉일 :  2019-05-30T00:00:00.000Z
제목 :  캡틴마블
장르 :  ['액션', '판타지']
개봉일 :  2019-03-06T00:00:00.000Z
제목 :  이터널스
장르 :  ['액션', '판타지']
개봉일 :  2020-12-01T00:00:00.000Z
제목 :  블랙위도우
장르 :  ['액션', 'SF']
개봉일 :  2020-04-28T00:00:00.000Z
제목 :  어벤져스 엔드게임
장르 :  ['액션', 'SF']
개봉일 :  2019-04-24T00:00:00.000Z
제목 :  여친남친
장르 :  ['로맨스', '멜로']
개봉일 :  2013-02-07T00:00:00.000Z
제목 :  남색대문
장르 :  ['로맨스', '멜로', '드라마']
개봉일 :  2002-02-20T00:00:00.000Z
제목 :  나의 EX
장르 :  ['로맨스', '멜로', '코미디', '드라마']
개봉일 :  2008-07-04T00:00:00.000Z
제목 :  하이자오 7번지
장르 :  ['로맨스', '멜로']
개봉일 :  2008-07-04T00:00:00.000Z
제목 :  카페 6
장르 :  ['로맨스', '멜로']
개봉일 :  2016-07-04T00:00:00.000Z


In [270]:
!curl -X POST "http://54.180.149.147:8080/api/team/2/movie" -H test

{"genres":[],"cast":[],"_id":"5e5c8acc168330138949b5c6","__v":0}

In [334]:
!curl -X GET "http://54.180.149.147:8080/api/team/2/movie" -H "accept: application/json"

[{"genres":["드라마"],"cast":["송강호","이선균"],"_id":"5e58dc10168330138949b5b3","title":"기생충","posterUrl":"https://movie-phinf.pstatic.net/20190528_36/1559024198386YVTEw_JPEG/movie_image.jpg","screeningStatus":"finished","advanceRate":12.34,"advanceRateRank":1,"visitorRating":5.67,"expertRating":5.67,"plot":" 폐 끼치고 싶진 않았어요 전원백수로 살 길 막막하지만 사이는 좋은 기택(송강호) 가족. 장남 기우(최우식)에게 명문대생 친구가 연결시켜 준 고액 과외 자리는 모처럼 싹튼 고정수입의 희망이다. 온 가족의 도움과 기대 속에박사장(이선균) 집으로 향하는 기우. 글로벌 IT기업 CEO인 박사장의 저택에 도착하자 젊고 아름다운 사모님 연교(조여정)가 기우를 맞이한다. 그러나 이렇게 시작된 두 가족의 만남 뒤로, 걷잡을 수 없는 사건이 기다리고 있었으니…","runtime":131,"released":"2019-05-30T00:00:00.000Z","director":"봉준호","__v":0},{"genres":["액션","판타지"],"cast":["브리라슨","사무엘 잭슨"],"_id":"5e58db9c168330138949b5b2","title":"캡틴마블","posterUrl":"https://movie-phinf.pstatic.net/20190225_72/1551069530582h2huX_JPEG/movie_image.jpg","screeningStatus":"finished","advanceRate":12.34,"advanceRateRank":1,"visitorRating":5.67,"expertRating":5.67,"plot":"위기에 빠진 어벤져스의 희망!1995년, 공군 파일럿 시절의 기억을 잃고 크리족 전사로 살아가던 

In [312]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

api_site_url = "http://54.180.149.147:8080/docs"

driver = wd.Chrome("./chromedriver")
driver.get(api_site_url)

driver.execute_script("window.scrollTo(0, 1000);")

driver.find_element_by_css_selector("div#operations-movie_2팀-post_team_2_movie").click()

driver.find_element_by_css_selector(".btn.try-out__btn").click()

json_area = driver.find_element_by_css_selector(".body-param__text")
# json_area.send_keys(Keys.SHIFT, Keys.HOME, Keys.BACK_SPACE)
# json_area.send_keys(Keys.BACK_SPACE)
json_area.clear()
json_area.send_keys(test)

driver.find_element_by_css_selector(".btn.execute.opblock-control__btn").click()

In [373]:
json_list[61]

'{"title": "몽마르트 파파", "posterUrl": "https://movie-phinf.pstatic.net/20191213_285/1576202593395PFG87_JPEG/movie_image.jpg", "screeningStatus": "open", "advanceRate": 0.01, "advanceRateRank": 62, "visitorRating": 5.0, "expertRating": 8.0, "plot": "중학교 미술교사로 평생을 살아온 아버지 어느 덧 정년 퇴임의 시간이 다가오고 문득 아버지의 정년 퇴임 이후가 궁금했던 아들은 질문을 하게 된다“아버지 앞으로 뭐 하실 거예요” “다 계획이 있지”놀랍게도 아버지의 계획은 파리의 몽마르트 언덕에서 그림을 그리는 것오랫동안 꿈꿔온 아버지의 꿈을 실현하기 위해 파리로 향하게 되는데 한 점 한 점 그림이 쌓이며 몽마르트 언덕의 한 명의 화가가 된 아버지“아버지 지금 기분이 어떠세요” “말해야 하나 끝내준다”당신의 꿈은 무엇이었습니까 오래된 꿈을 현실로", "genres": ["다큐멘터리"], "runtime": 83, "cast": ["민형식", "이운숙", "민병우"], "released": " 2020-01-09", "director": "민병우"}'

### 셀레니움을 활용해 자동으로 추가하기

In [374]:
from selenium import webdriver as wd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
import time


for i in range(len(json_list)):
    api_site_url = "http://54.180.149.147:8080/docs"

    driver = wd.Chrome("./chromedriver")
    driver.get(api_site_url)
    
    time.sleep(2.0)

    driver.execute_script("window.scrollTo(0, 1000);")

    driver.find_element_by_css_selector("div#operations-movie_2팀-post_team_2_movie").click()

    driver.find_element_by_css_selector(".btn.try-out__btn").click()

    json_area = driver.find_element_by_css_selector(".body-param__text")
    json_area.clear()
    json_area.send_keys(json_list[i])

    driver.find_element_by_css_selector(".btn.execute.opblock-control__btn").click()
    json_area.clear()
    
    driver.close()